In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
from scipy import ndimage, sparse
from scipy.linalg import eigh, inv, logm, norm
import scipy.sparse
import numpy as np
import matplotlib.pyplot as plt
import sys
import glob
from IPython.display import clear_output
import networkx as nx
import seaborn as sns; sns.set()

In [2]:
# In[2]:
def edges_rescaling(edges,scale): # edges are mat.data where mat is a sparse scipy matrix
    edges = np.log10(edges) # log rescale weights because they vary over many decades
    edges -= min(edges) # make them positive 
    edges /= max(edges)*1.0/scale # rescale from 0 to scale
    return edges
def build_omegaij(Kdata,Krow,Kcol,m):
    omegaIJ_data = np.zeros(Kdata.shape)
    omegaIJ_data = np.asfarray([Kdata[ind]*(1.0/m[Krow[ind]] + 1.0/m[Kcol[ind]]) for ind in range(omegaIJ_data.shape[0])])
    omegaIJ = sparse.coo_matrix((omegaIJ_data, (Krow, Kcol)), shape=(Krow.max()+1,Kcol.max()+1))
    return omegaIJ
def build_omegai(K,m):
    omegaI = 0.5*np.divide(K.sum(axis=1),m.reshape((m.shape[0],1)))     #0.5 to avoid double counting
    return omegaI
def remove_col(mat,index_to_drop): #csr
    to_keep = list(set(range(mat.shape[1]))-set(index_to_drop))    
    mat = mat[:,to_keep]
    return mat
def remove_row(mat,index_to_drop): #csc
    to_keep = list(set(range(mat.shape[0]))-set(index_to_drop))    
    mat = mat[to_keep,:]
    return mat
def remove_2nodes(mat,nodes):
    mat = mat.tocoo()
    todrop1 = np.logical_or((mat.row==nodes[0]),(mat.row==nodes[1])).nonzero()[0]
    todrop2 = np.logical_or((mat.col==nodes[0]),(mat.col==nodes[1])).nonzero()[0]
    todrop = list(set(np.concatenate((todrop1,todrop2))))
    newdata=np.delete(mat.data,todrop)
    newrow=np.delete(mat.row,todrop)
    newcol=np.delete(mat.col,todrop)
    return sparse.coo_matrix((newdata, (newrow, newcol)), shape=mat.shape)
def remove_1node(mat,node):
    mat = mat.tocoo()
    todrop = np.logical_or((mat.row==node[0]),(mat.col==node[0])).nonzero()[0]
    todrop = list(set(todrop))
    newdata=np.delete(mat.data,todrop)
    newrow=np.delete(mat.row,todrop)
    newcol=np.delete(mat.col,todrop)
    return sparse.coo_matrix((newdata, (newrow, newcol)), shape=mat.shape)
def expand(Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol,idxs,m,g):
    for idx in idxs:
        newdata=K.data[idx]
        j=K.col[idx]
        Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol = expand1(Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol,newdata,m,g,j)
    return Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol
def expand1(Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol,newk,m,i,j):
    #add (i,j)
    Kdata = np.append(Kdata,newk)
    Krow = np.append(Krow,i)
    Kcol = np.append(Kcol,j)
    omegaIJdata = np.append(omegaIJdata,newk*(1.0/m[i]+1.0/m[j]))
    omegaIJrow = np.append(omegaIJrow,i)
    omegaIJcol = np.append(omegaIJcol,j)
    #add (j,i)
    Kdata = np.append(Kdata,newk)
    Krow = np.append(Krow,j)
    Kcol = np.append(Kcol,i)
    omegaIJdata = np.append(omegaIJdata,newk*(1.0/m[i]+1.0/m[j]))
    omegaIJrow = np.append(omegaIJrow,j)
    omegaIJcol = np.append(omegaIJcol,i)
    return Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol
def delete_nodes(Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol,idxs): #this is not symm wrt to (i,j)
    Kdata = np.delete(Kdata,idxs)
    Krow = np.delete(Krow,idxs)
    Kcol = np.delete(Kcol,idxs)
    omegaIJdata = np.delete(omegaIJdata,idxs)
    omegaIJrow = np.delete(omegaIJrow,idxs)
    omegaIJcol = np.delete(omegaIJcol,idxs)
    return Kdata,Krow,Kcol,omegaIJdata,omegaIJrow,omegaIJcol

In [9]:
import pickle
from functools import reduce

In [196]:
with open('./communities.txt', 'rb') as f:
    communities = pickle.load(f)
c = communities[1]
print('size of the community: '+str(len(c)))

size of the community: 10014


In [202]:
#Restrict the graph to the community:
K0_XY = sparse.load_npz('/home/garner1/Work/pipelines/tissue2graph/npz/covd_mat_XY_nn19.npz')
pos = np.load('/home/garner1/Work/pipelines/tissue2graph/npz/covd_X-XY_data.npz',allow_pickle=True)['XY'][c]
K = K0_XY[c,:]
K = K.tocsc()[:,c].tocoo()
m = np.ones(K.shape[0]) #initial masses

#Initialize omegaIJ/I
omegaIJ = build_omegaij(K.data,K.row,K.col,m)
omegaI = build_omegai(K,m) 
N = np.count_nonzero(omegaI)

In [203]:
#Delete light edges to speedup computation:
threshold = 1.0e-1
idxs_todelete = np.where(np.abs(K.data) < threshold)[0]
print(K.data.shape)
K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,idxs_todelete)
print(K.data.shape)

(192672,)
(104446,)


In [204]:
sns.set(style='white', rc={'figure.figsize':(50,50)})
mat = sparse.coo_matrix((K.data, (K.row, K.col)), shape=(K.row.max()+1, K.col.max()+1)) 
G = nx.from_scipy_sparse_matrix(mat) # if sparse matrix
eset = [(u, v) for (u, v, d) in G.edges(data=True)]
weights = [d['weight'] for (u, v, d) in G.edges(data=True)]
nx.draw_networkx_nodes(G, pos,alpha=0.0)
nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=1.0, width=weights,edge_color='r',style='solid')

plt.axis('off')
plt.savefig('./before_RG.png',bbox_inches='tight')
plt.close()

/usr/local/share/anaconda3/lib/python3.7/site-packages/networkx/drawing/nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


In [205]:
# In[54]:
'''RG flow'''
condition = True
Imax0 = 0
IJmax0 = 0
counter = 0
while condition:
    counter += 1
    #Find max btw node and edges:
    IJmax_idx = np.where( omegaIJ.data==np.max(omegaIJ.data[np.nonzero(omegaIJ.data)]) )[0][0] #idx of max nonzero edge
    IJmax = omegaIJ.data[IJmax_idx] #value of max nonzero edge
    i0 = np.where( omegaI==np.max(omegaI[np.nonzero(omegaI)]) )[0][0] #label of max nonzero node connectivity
    Imax = omegaI[i0][0,0] #value of max nonzero node connectivity
    maxtype = np.argmax([Imax,IJmax]) # max btw node and edge

    #Set the condition for the loop:
    condition = (abs(Imax-Imax0) > 1e-6) or (abs(IJmax-IJmax0) > 1e-6); Imax0 = Imax; IJmax0 = IJmax
#     condition = np.count_nonzero(omegaI) > N/10

#     clear_output()
    if Imax<=IJmax: #if edge
        if (counter == 100): #print every 100 steps
            print('edge removal')
            print(counter,[Imax,IJmax],np.count_nonzero(omegaI),omegaIJ.data.shape[0])
            counter = 0
            
        i0 = K.row[IJmax_idx];j0 = K.col[IJmax_idx] #find max edge (i0,j0) nodes
        m = np.append(m,m[i0]+m[j0]) #add a center of mass node
        g = K.row.max()+1 # label the i0-j0 center of mass node
        pos = np.vstack((pos,[0.5*(pos[i0,0]+pos[j0,0]),0.5*(pos[i0,1]+pos[j0,1])])) #set the coord of the cent of mass
        idx_i0isrow = np.argwhere(K.row==i0) # idxs of (i0,.)
        idx_i0iscol = np.argwhere(K.col==i0) # idxs of (.,i0)
        idx_j0isrow = np.argwhere(K.row==j0) # idxs of (j0,.)
        idx_j0iscol = np.argwhere(K.col==j0) # idxs of (.,j0)
        js = np.setdiff1d([K.col[idx] for idx in np.union1d(idx_i0isrow,idx_j0isrow)],[i0,j0]) #nodes neighbours of i0 and j0
        for j in  js: #for any node in the nn of i0 and j0
            idx_i0j = np.intersect1d(idx_i0isrow,np.argwhere(K.col==j)) #(i0,j)
            idx_j0j = np.intersect1d(idx_j0isrow,np.argwhere(K.col==j)) #(j0,j)
            #add (g,j) to the graph
            newk = np.sum(np.append(K.data[idx_i0j],K.data[idx_j0j]))
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = expand1(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,newk,m,g,j)
            #remove i0 and j0 from K, omegaIJ
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,np.intersect1d(np.argwhere(K.row==i0),np.argwhere(K.col==j)))
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,np.intersect1d(np.argwhere(K.row==j),np.argwhere(K.col==i0)))
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,np.intersect1d(np.argwhere(K.row==j0),np.argwhere(K.col==j)))
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,np.intersect1d(np.argwhere(K.row==j),np.argwhere(K.col==j0)))
            #remove (i0,j0) from K, omegaIJ
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,np.intersect1d(np.argwhere(K.row==i0),np.argwhere(K.col==j0)))
            K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,np.intersect1d(np.argwhere(K.row==j0),np.argwhere(K.col==i0)))
        #update omegaI
        omegaI_g = np.array(sum([K.data[idx] for idx in np.argwhere(K.row==g)])*1.0/m[g]).reshape(1,1)
        omegaI = np.append(omegaI,omegaI_g,0)
        for j in js: #for any node in the nn of i0 and j0
            omegaI[j] = sum([K.data[idx] for idx in np.argwhere(K.row==j)])*1.0/m[j]
        omegaI[i0] = 0.0; omegaI[j0] = 0.0
        
    if Imax>IJmax: #if node
        if (counter == 100):
            print('node removal')
            print(counter,[Imax,IJmax],np.count_nonzero(omegaI),omegaIJ.data.shape[0])
            counter = 0
            
        idx_i0isrow = np.argwhere(K.row==i0) # idxs of (i0,.)
        idx_i0iscol = np.argwhere(K.col==i0) # idx of (.,i0)
        js = np.unique(K.col[idx_i0isrow]) # nn j in (i0,.)
        connectivity = omegaI[i0]*m[i0]
        for i in js: #for any node in the nn of i0
            idx_ri = np.argwhere(K.row==i) #(i,.)
            idx_ci = np.argwhere(K.col==i) #(.,i)
            for j in js[np.argwhere(js==i)[0][0]+1:]:
                idx_cj = np.argwhere(K.col==j) #(.j)
                idx_rj = np.argwhere(K.row==j) #(j,.)
                idx_ij = np.intersect1d(idx_ri,idx_cj) #(i,j)
                idx_ji = np.intersect1d(idx_rj,idx_ci) #(j,i)
                idx_ii0 = np.intersect1d(idx_ri,idx_i0iscol) #(i,i0)   
                idx_i0j = np.intersect1d(idx_i0isrow,idx_cj) #(i0,j)
                if idx_ij.shape[0]>0: #update edge value if it exists, without the need to create new .row and .col
                    K.data[idx_ij] = np.sum(np.append(K.data[idx_ij],K.data[idx_ii0]*K.data[idx_i0j]/connectivity)) #(i,j)
                    K.data[idx_ji] = K.data[idx_ij] #(j,i)
                    omegaIJ.data[idx_ij] = K.data[idx_ij]*(1.0/m[i]+1.0/m[j]) 
                    omegaIJ.data[idx_ij] = omegaIJ.data[idx_ji]
                if idx_ij.shape[0]==0: #create a new edge
                    newk = K.data[idx_ii0]*K.data[idx_i0j]/connectivity
                    if newk >= 0.25*threshold: #set a threshold to avoid a dense graphs being generated in the flow
                        K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = expand1(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,newk,m,i,j)
        #remove i0 from K, omegaIJ
        K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,np.argwhere(K.row==i0))
        K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col = delete_nodes(K.data,K.row,K.col,omegaIJ.data,omegaIJ.row,omegaIJ.col,np.argwhere(K.col==i0))
        #update omegaI
        for j in js: #for any node in the nn of i0
            omegaI[j] = sum([K.data[idx] for idx in np.argwhere(K.row==j)])*1.0/m[j]
        omegaI[i0] = 0.0  #set i0 to 0 in omegaI
        # pos[i0,0] = np.mean(pos[:,0])
        # pos[i0,1] = np.mean(pos[:,1])

node removal
100 [6.318915459020732, 2.0] 9915 103582
node removal
100 [5.890949740248548, 2.0] 9815 102718
node removal
100 [5.647683810714111, 2.0] 9715 101900
node removal
100 [5.735436370183902, 2.0] 9615 101030
node removal
100 [5.601208148417195, 2.0] 9515 100154
node removal
100 [5.505537965223649, 2.0] 9415 99418
node removal
100 [5.4938190491259045, 2.0] 9315 98562
node removal
100 [5.413279924189994, 2.0] 9215 97840
node removal
100 [5.176310128859293, 2.0] 9115 96958
node removal
100 [5.803915527801392, 2.0] 9015 95882
node removal
100 [5.599690052180091, 2.0] 8915 95188
node removal
100 [5.208947992317564, 2.0] 8815 94328
node removal
100 [6.313683340492302, 2.0] 8715 93476
node removal
100 [5.437693296678296, 2.0] 8615 92592
node removal
100 [6.265429981418163, 2.0] 8515 91728
node removal
100 [5.863152519243526, 2.0] 8415 90894
node removal
100 [6.635395354695835, 2.0] 8315 90032
node removal
100 [5.502674699529392, 2.0] 8215 89174
node removal
100 [5.2039545799625815, 2.

In [212]:
fixed_mat = sparse.coo_matrix((K.data, (K.row, K.col)), shape=(K.row.max()+1, K.col.max()+1))
sns.set(style='white', rc={'figure.figsize':(50,50)})
G = nx.from_scipy_sparse_matrix(fixed_mat) 
eset = [(u, v) for (u, v, d) in G.edges(data=True)]
weights = [d['weight'] for (u, v, d) in G.edges(data=True)]
nx.draw_networkx_nodes(G, pos,alpha=0.0)
nx.draw_networkx_edges(G, pos, edgelist=eset,alpha=1.0, width=weights,edge_color='r',style='solid')

plt.axis('off')
plt.title('(nodes,edges): '+str(np.count_nonzero(omegaI))+'; '+str(len(eset)),fontsize=100)
plt.savefig('./after_RG.png',bbox_inches='tight')
plt.close()